<a href="https://colab.research.google.com/github/eduardarsimoes/lae1/blob/main/Titanic/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DATASET TITANIC**
---
### **LAB. EXTENSÃO 2020/2 EAD** - Eduarda Simões, Serenna Ferrari e Thaís de Souza

### Importando bibliotecas e respectivos módulos

In [1]:
import sys

!"{sys.executable}" -m pip install -U pandas-profiling[notebook]
!jupyter nbextension enable --py widgetsnbextension

     |████████████████████████████████| 245kB 5.7MB/s 
     |████████████████████████████████| 61kB 4.5MB/s 
     |████████████████████████████████| 3.2MB 7.2MB/s 
     |████████████████████████████████| 1.1MB 29.0MB/s 
     |████████████████████████████████| 81kB 7.1MB/s 
     |████████████████████████████████| 81kB 7.5MB/s 
     |████████████████████████████████| 122kB 40.2MB/s 
     |████████████████████████████████| 296kB 35.4MB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp37-none-any.whl size=27085 sha256=1ee076abc907d3e8b0abdcae79c1eaa07e603208951ffe40d0281627aca23e0f
  Stored in directory: /root/.cache/pip/wheels/43/07/ac/7c5a9d708d65247ac1f94066cf1db075540b85716c30255459
  Created wheel for phik: filename=phik-0.11.2-cp37-none-any.whl size=1107413 sha256=ce34b7df1c997c4c44b56a24c225b6d1e3f978412ca4ddf6af30dd14bc35e308
  Stored in directory: /root/.cache/pip/wheels/c0/a3/b0/f27b1cfe32ea131a3715169132ff6d85653789e80e966c3bf6
Successfully built htmlmin phik
ERROR: goo

In [2]:
# Importando bibliotecas
import sklearn as skl
import pandas as pd
import seaborn as sns
import matplotlib as plt
import numpy as np
import scipy as sc

#from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from pandas_profiling import ProfileReport


### Carregando dataset e resumo de dados

In [3]:
url = "https://raw.githubusercontent.com/profmoisesomena/escience_and_tools/master/data/titanic_dados_t.csv"
#RENOMEOU-SE ALGUNS ATRIBUTOS
names = ["PassengerId","survived","shipClass","name","sex","age","siblingsSpousesOnboard","parentsChildrenOnboard","ticket","fareTicket","cabin","embarkedHarbor"]
dataset = pd.read_csv(url, header=0, names=names)

pd.pandas.set_option('display.max_columns', None)
print(dataset.head())


   PassengerId  survived  shipClass  \
0            1         0          3   
1            2         1          1   
2            3         1          3   
3            4         1          1   
4            5         0          3   

                                                name     sex   age  \
0                            Braund, Mr. Owen Harris    male  22.0   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0   
2                             Heikkinen, Miss. Laina  female  26.0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0   
4                           Allen, Mr. William Henry    male  35.0   

   siblingsSpousesOnboard  parentsChildrenOnboard            ticket  \
0                       1                       0         A/5 21171   
1                       1                       0          PC 17599   
2                       0                       0  STON/O2. 3101282   
3                       1                       0           

### Pré processamento

In [4]:
#VERIFICANDO DADOS DUPLICADOS
sum(dataset.duplicated())

0

In [5]:
#REMOVENDO ATRIBUTOS IRRELEVANTES
dataset.drop(["PassengerId","name","ticket","cabin"], axis = 1, inplace = True) #axis = 1 (toda coluna) | inplace = True (replace)
print(dataset.head())

   survived  shipClass     sex   age  siblingsSpousesOnboard  \
0         0          3    male  22.0                       1   
1         1          1  female  38.0                       1   
2         1          3  female  26.0                       0   
3         1          1  female  35.0                       1   
4         0          3    male  35.0                       0   

   parentsChildrenOnboard  fareTicket embarkedHarbor  
0                       0      7.2500              S  
1                       0     71.2833              C  
2                       0      7.9250              S  
3                       0     53.1000              S  
4                       0      8.0500              S  


In [6]:
#VERIFICANDO OS DADOS NULOS NO CONJUNTO DE TREINO
dataset_analise_nulos = dataset
stat_data = dataset_analise_nulos.describe().T #T = transpose
stat_data['nulos'] = pd.DataFrame(dataset_analise_nulos.isnull().sum())
stat_data['percentualNulos'] = pd.DataFrame(dataset_analise_nulos.isna().mean().round(4)*100)
stat_data['type'] = pd.Series(dataset_analise_nulos.dtypes)
stat_data


,count,mean,std,min,25%,50%,75%,max,nulos,percentualNulos,type
survived,891.0,0.383838,0.486592,0.00,0.0000,0.0000,1.0,1.0000,0,0.00,int64
shipClass,891.0,2.308642,0.836071,1.00,2.0000,3.0000,3.0,3.0000,0,0.00,int64
age,714.0,29.699118,14.526497,0.42,20.1250,28.0000,38.0,80.0000,177,19.87,float64
siblingsSpousesOnboard,891.0,0.523008,1.102743,0.00,0.0000,0.0000,1.0,8.0000,0,0.00,int64
parentsChildrenOnboard,891.0,0.381594,0.806057,0.00,0.0000,0.0000,0.0,6.0000,0,0.00,int64
fareTicket,891.0,32.204208,49.693429,0.00,7.9104,14.4542,31.0,512.3292,0,0.00,float64


In [7]:
 #PREENCHENDO OS VALORES NULOS -age-
dataset['age'].fillna(method='ffill', inplace=True) # Valor linha anterior
dataset

,survived,shipClass,sex,age,siblingsSpousesOnboard,parentsChildrenOnboard,fareTicket,embarkedHarbor
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,19.0,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [8]:
#VERIFICANDO OS VALORES NULOS TRATADOS ANTERIORMENTE
dataset.isnull().sum()

survived                  0
shipClass                 0
sex                       0
age                       0
siblingsSpousesOnboard    0
parentsChildrenOnboard    0
fareTicket                0
embarkedHarbor            2
dtype: int64

In [9]:
#PREENCHENDO OS VALORES NULOS -embarkedHarbor-
dataset['embarkedHarbor'].fillna(method='ffill', inplace=True) # Valor linha anterior
dataset

,survived,shipClass,sex,age,siblingsSpousesOnboard,parentsChildrenOnboard,fareTicket,embarkedHarbor
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,19.0,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [10]:
#VERIFICANDO OS VALORES NULOS TRATADOS ANTERIORMENTE
dataset.isnull().sum()

survived                  0
shipClass                 0
sex                       0
age                       0
siblingsSpousesOnboard    0
parentsChildrenOnboard    0
fareTicket                0
embarkedHarbor            0
dtype: int64

In [11]:
#VERIFICANDO DADOS DUPLICADOS
sum(dataset.duplicated())

58

In [ ]:
#CONVERTENDO OS VALORES DO ATRIBUTO 'SEX' - female, male PARA 0, 1

## label_encoder object knows how to understand word labels. 
#label_encoder = preprocessing.LabelEncoder()
## Encode labels in column 'sex'. 
#dataset['sex']= label_encoder.fit_transform(dataset['sex']) 

In [12]:
#ONE HOT ENCONDING - convertendo os dados categóricos em dados numéricos (não ordinais)
dataset = pd.get_dummies(dataset)
print(dataset.head())

   survived  shipClass   age  siblingsSpousesOnboard  parentsChildrenOnboard  \
0         0          3  22.0                       1                       0   
1         1          1  38.0                       1                       0   
2         1          3  26.0                       0                       0   
3         1          1  35.0                       1                       0   
4         0          3  35.0                       0                       0   

   fareTicket  sex_female  sex_male  embarkedHarbor_C  embarkedHarbor_Q  \
0      7.2500           0         1                 0                 0   
1     71.2833           1         0                 1                 0   
2      7.9250           1         0                 0                 0   
3     53.1000           1         0                 0                 0   
4      8.0500           0         1                 0                 0   

   embarkedHarbor_S  
0                 1  
1                 0  
2 

In [13]:
#ENCAIXOTANDO (BINNING) PREÇOS DAS TARIFAS
valuesFare = dataset.fareTicket
# 'fareTicket_very_low' = 1,'fareTicket_low' = 2, 'fareTicket_mid' = 3, 'fareTicket_high' = 4,'fareTicket_very_high' = 5
groups_names = [1, 2, 3, 4, 5]
categories = pd.qcut(valuesFare, 5, labels=groups_names)
pd.value_counts(categories)
dataset['categories_fareTicket'] = categories
#dataset.drop("fareTicket", axis = 1, inplace = True)
dataset

,survived,shipClass,age,siblingsSpousesOnboard,parentsChildrenOnboard,fareTicket,sex_female,sex_male,embarkedHarbor_C,embarkedHarbor_Q,embarkedHarbor_S,categories_fareTicket
0,0,3,22.0,1,0,7.2500,0,1,0,0,1,1
1,1,1,38.0,1,0,71.2833,1,0,1,0,0,5
2,1,3,26.0,0,0,7.9250,1,0,0,0,1,2
3,1,1,35.0,1,0,53.1000,1,0,0,0,1,5
4,0,3,35.0,0,0,8.0500,0,1,0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,27.0,0,0,13.0000,0,1,0,0,1,3
887,1,1,19.0,0,0,30.0000,1,0,0,0,1,4
888,0,3,19.0,1,2,23.4500,1,0,0,0,1,4
889,1,1,26.0,0,0,30.0000,0,1,1,0,0,4


In [ ]:
''' #EXCLUINDO OUTLIERS
dataset_summary = dataset.iloc[:,:-1]
q1 = dataset_summary.quantile(0.25)
q3 = dataset_summary.quantile(0.75)
IQR = q3 - q1
lowqe_bound = q1 - 1.5 * IQR
upper_bound = q3 + 1.5 * IQR
print(lowqe_bound, upper_bound) '''

survived                  -1.500
shipClass                  0.500
age                       -7.000
siblingsSpousesOnboard    -1.500
parentsChildrenOnboard     0.000
fareTicket               -26.724
sex_female                -1.500
sex_male                  -1.500
embarkedHarbor_C           0.000
embarkedHarbor_Q           0.000
embarkedHarbor_S          -1.500
dtype: float64 survived                   2.5000
shipClass                  4.5000
age                       65.0000
siblingsSpousesOnboard     2.5000
parentsChildrenOnboard     0.0000
fareTicket                65.6344
sex_female                 2.5000
sex_male                   2.5000
embarkedHarbor_C           0.0000
embarkedHarbor_Q           0.0000
embarkedHarbor_S           2.5000
dtype: float64


In [ ]:
''' IQR_dataset = dataset_summary[~((dataset_summary < lowqe_bound) | (dataset_summary > upper_bound)).any(axis=1)]
IQR_dataset.shape '''

(451, 11)

In [14]:
#BALANCEAMENTO = VERIFICANDO VARIAÇÃO NO ATRIBUTO ALVO
target = "survived"
count_class_0, count_class_1 = dataset[target].value_counts()
print(count_class_0)
print(count_class_1)

549
342


### Separando conjuntos de dados para treinamento e teste

In [15]:
array = dataset.values 
x, y = array[:, 1:], array[:,0]

In [16]:
x_train, x_teste, y_train, y_teste = train_test_split(x, y, test_size = 0.25, random_state = 0, shuffle = True)

In [17]:
model = LogisticRegression() # CLASSIFICACAO
model.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [18]:
print("\n --- Precisão de treino --- \n ==> ", skl.metrics.accuracy_score(y_train, model.predict(x_train)))
print("\n --- Precisão de teste ---\n ==>", skl.metrics.accuracy_score(y_teste, model.predict(x_teste)))


 --- Precisão de treino --- 
 ==>  0.7964071856287425

 --- Precisão de teste ---
 ==> 0.8026905829596412


In [19]:
# Profiling Report
profile = ProfileReport(dataset, html={"style": {"full_width": True}}, sort="None")
profile.to_file("output.html")

Summarize dataset:   0%|          | 0/25 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]